In [1]:
import numpy as np

In [2]:
def matrix_to_string(matrix):
    # Converting each row of the matrix to a string
    row_strings = [" ".join(f"{value:.16f}" for value in row) for row in matrix]

    # Joining all row strings with a newline character
    return "\n".join(row_strings)

In [3]:
def string_to_matrix(string):
    string = [list(map(float, row.split())) for row in string.strip().split('\n')]

    # Converting the data to a NumPy matrix
    return np.array(string)

In [4]:
def cell_expand(primitive_lattice):
    transformation_matrix = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 2]
    ])
    #def expand_lattice_param(primitive_lattice, transformation_matrix):
    primitive_lattice = string_to_matrix(primitive_lattice)

    # Performing the matrix multiplication
    lattice_basis = np.dot(transformation_matrix, primitive_lattice)

    # Convert the matrix to a string in the desired format
    formatted_string = matrix_to_string(lattice_basis)

    print(formatted_string)
    return lattice_basis



In [5]:
# Transformation matrix t
transformation_matrix = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 2]
])

def transformation(data_str):
    # Splitting the data into rows, separating labels from numerical data
    labels, numerical_data = [], []
    for row in data_str.strip().split('\n'):
        elements = row.split()
        labels.append(elements[0])  # Extract label
        numerical_data.append(list(map(float, elements[1:])))  # Convert remaining elements to floats

    # Converting the numerical data to a NumPy matrix
    matrix = np.array(numerical_data)

    # Calculating the inverse of the transformation matrix
    inverse_t = np.linalg.inv(transformation_matrix)

    # Performing the matrix multiplication
    result = np.dot(inverse_t, matrix.T).T

    # Transpose back and reattach the labels
    result_labeled = np.column_stack((labels, result))
    return result, result_labeled

def shifting(matrix):
    # Apply the first transformation (add 0.5 to the third column)
    transformed_matrix_z = np.copy(matrix)
    transformed_matrix_z[:, 2] += 0.5  # Adjusted index for mixed type

    # Intertwine the rows from the first transformation
    intertwined_matrix_z = np.vstack([np.vstack([row, transformed_row]) for row, transformed_row in zip(matrix, transformed_matrix_z)])

    # Apply the second transformation (add 0.5 to the first and second columns)
    def second_transformation(row):
        new_row = np.copy(row)
        new_row[0] += 0.5  # Adjusted index for mixed type
        new_row[1] += 0.5  # Adjusted index for mixed type
        return new_row

    intertwined_matrix_xy = np.vstack([np.vstack([row, second_transformation(row)]) for row in intertwined_matrix_z])
    intertwined_matrix_xy = intertwined_matrix_z%1
    return intertwined_matrix_xy


def expand_positions(data_str):
    result, result_labeled = transformation(data_str)

    label_to_data = {}

    for row in result_labeled:
        elements = row
        label = elements[0]  # Extract label
        numerical_values = list(map(float, elements[1:]))  # Convert remaining elements to floats
        # Add numerical data to corresponding label list
        if label in label_to_data:
            label_to_data[label].append(numerical_values)
        else:
            label_to_data[label] = [numerical_values]

    # Print unique labels
    print("Unique Labels:", label_to_data.keys())


    # Apply the transformation and intertwine the rows
    unlabled_positions = np.vstack([shifting(label_to_data[lables]) for lables in label_to_data.keys()])


    # Apply the transformation and intertwine the rows
    transformed_arrays = []
    for label in label_to_data.keys():
        transformed_data = shifting(label_to_data[label])

        # Prepend each row with the label
        labeled_data = np.hstack([[[label]] * transformed_data.shape[0], transformed_data])  # Repeat label for each row
        transformed_arrays.append(labeled_data)

    # Stack the transformed arrays vertically
    labled_positions = np.vstack(transformed_arrays)

    # Convert the matrix to a string in the desired format
    formatted_num = matrix_to_string(unlabled_positions)
    print(formatted_num)
    return unlabled_positions, labled_positions

In [6]:
# Input data as a multiline string
NaNNO_prim_lattice = """
3.8280824710192780    0.0000000000000000    0.0000000000000000
0.0000000000000000    3.8280824710192780    0.0000000000000000
0.0000000000000000    0.0000000000000000   10.9441331724216901
"""
NaNNO_super_lattice = cell_expand(NaNNO_prim_lattice)

3.8280824710192780 0.0000000000000000 0.0000000000000000
0.0000000000000000 3.8280824710192780 0.0000000000000000
0.0000000000000000 0.0000000000000000 21.8882663448433803


In [25]:
# Input data as a multiline string, including labels
NaNNO_prim_positions = """
Na  0.5000000000000000  0.5000000000000000  0.0000000000000000
Nd  0.5000000000000000  0.5000000000000000  0.5000000000000000
Nb  0.0000000000000000  0.0000000000000000  0.2935347165073026
Nb  0.0000000000000000  0.0000000000000000  0.7064652834926974
O  0.0000000000000000  0.0000000000000000  0.5000000000000000
O  0.5000000000000000  0.0000000000000000  0.6645394444672732
O  0.0000000000000000  0.5000000000000000  0.6645394444672732
O  0.0000000000000000  0.5000000000000000  0.3354605255327243
O  0.5000000000000000  0.0000000000000000  0.3354605255327243
O  0.0000000000000000  0.0000000000000000  0.1299886821043827
O  0.0000000000000000  0.0000000000000000  0.8700113178956173
"""

NaNNO_super_unlabled_positions, NaNNO_super_labled_positions = expand_positions(NaNNO_prim_positions)

Unique Labels: dict_keys(['Na', 'Nd', 'Nb', 'O'])
0.5000000000000000 0.5000000000000000 0.0000000000000000
0.5000000000000000 0.5000000000000000 0.5000000000000000
0.5000000000000000 0.5000000000000000 0.2500000000000000
0.5000000000000000 0.5000000000000000 0.7500000000000000
0.0000000000000000 0.0000000000000000 0.1467673582536513
0.0000000000000000 0.0000000000000000 0.6467673582536513
0.0000000000000000 0.0000000000000000 0.3532326417463487
0.0000000000000000 0.0000000000000000 0.8532326417463487
0.0000000000000000 0.0000000000000000 0.2500000000000000
0.0000000000000000 0.0000000000000000 0.7500000000000000
0.5000000000000000 0.0000000000000000 0.3322697222336366
0.5000000000000000 0.0000000000000000 0.8322697222336366
0.0000000000000000 0.5000000000000000 0.3322697222336366
0.0000000000000000 0.5000000000000000 0.8322697222336366
0.0000000000000000 0.5000000000000000 0.1677302627663622
0.0000000000000000 0.5000000000000000 0.6677302627663622
0.5000000000000000 0.0000000000000000 

In [26]:
def transform_array(arr):
    # Ensure the input is a 3xN array
    if arr.shape[1] != 3:
        raise ValueError("Input array must have 3 rows")

    # Find columns where the value in the third column is greater than 0.5
    condition = arr[:, 2] > 0.48

    # Add 0.5 to the values in the first column for the selected rows
    arr[condition,0] += 0.5
    arr[condition,1] += 0.5

    # Apply modulus 1 to all values in the first column
    arr[:, 0] %= 1
    arr[:, 1] %= 1

    return arr

NaNNO_Cmcm =matrix_to_string(transform_array(NaNNO_super_unlabled_positions))
print(NaNNO_Cmcm, "\n")

0.5000000000000000 0.5000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.5000000000000000
0.5000000000000000 0.5000000000000000 0.2500000000000000
0.0000000000000000 0.0000000000000000 0.7500000000000000
0.0000000000000000 0.0000000000000000 0.1467673582536513
0.5000000000000000 0.5000000000000000 0.6467673582536513
0.0000000000000000 0.0000000000000000 0.3532326417463487
0.5000000000000000 0.5000000000000000 0.8532326417463487
0.0000000000000000 0.0000000000000000 0.2500000000000000
0.5000000000000000 0.5000000000000000 0.7500000000000000
0.5000000000000000 0.0000000000000000 0.3322697222336366
0.0000000000000000 0.5000000000000000 0.8322697222336366
0.0000000000000000 0.5000000000000000 0.3322697222336366
0.5000000000000000 0.0000000000000000 0.8322697222336366
0.0000000000000000 0.5000000000000000 0.1677302627663622
0.5000000000000000 0.0000000000000000 0.6677302627663622
0.5000000000000000 0.0000000000000000 0.1677302627663622
0.0000000000000000 0.5000000000